In [1]:
%matplotlib inline
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T    

from Fairness_Metrics import fairness_metric

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('csv/IA_counties.csv')
data.head()

,X,Y,STATEFP10,COUNTYFP10,GEOID10,NAME10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,...,PRES08OTH,TOTVOT12,PRES12D,PRES12R,PRES12OTH,TOTVOT16,PRES16D,PRES16R,PRES16OTH,CD
0,-92.999001,42.035801,19,127,19127,Marshall,Marshall County,1482770678,1803086,42.041691,...,356,19064,10257,8472,335,17980,7652,9146,1182,1
1,-92.066966,42.080299,19,11,19011,Benton,Benton County,1855117342,5760770,42.092547,...,207,14023,6862,6940,221,13844,4678,8232,934,1
2,-95.150846,43.082520,19,41,19041,Clay,Clay County,1469139214,13866941,43.079822,...,118,8502,3385,4951,166,8617,2249,5877,491,4
3,-95.325464,41.684636,19,165,19165,Shelby,Shelby County,1530110414,1486135,41.679014,...,102,6483,2469,3911,103,6370,1662,4362,346,4
4,-91.352586,42.862146,19,43,19043,Clayton,Clayton County,2016405612,36586071,42.840998,...,143,9138,4806,4164,168,9129,3237,5317,575,1


In [4]:
data.columns

Index(['X', 'Y', 'STATEFP10', 'COUNTYFP10', 'GEOID10', 'NAME10', 'NAMELSAD10',
       'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'TOTPOP', 'NH_WHITE',
       'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE',
       'HISP', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'N_NHPI', 'H_OTHER',
       'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP',
       'NHPIVAP', 'OTHERVAP', '2MOREVAP', 'TOTVOT00', 'PRES00D', 'PRES00R',
       'PRES00G', 'PRES00OTH', 'TOTVOT04', 'PRES04D', 'PRES04R', 'PRES04OTH',
       'TOTVOT08', 'PRES08D', 'PRES08R', 'PRES08OTH', 'TOTVOT12', 'PRES12D',
       'PRES12R', 'PRES12OTH', 'TOTVOT16', 'PRES16D', 'PRES16R', 'PRES16OTH',
       'CD'],
      dtype='object')

In [5]:
data[['TOTVOT16', 'PRES16D', 'PRES16R']]

,TOTVOT16,PRES16D,PRES16R
0,17980,7652,9146
1,13844,4678,8232
2,8617,2249,5877
3,6370,1662,4362
4,9129,3237,5317
...,...,...,...
94,16151,6215,8803
95,50667,25709,19458
96,6872,1856,4527
97,42680,15355,24447


In [6]:
req_data = pd.DataFrame({})
req_data['d_votes'] = data['PRES16D'] / data['TOTVOT16']
req_data['r_votes'] = data['PRES16R'] / data['TOTVOT16']
req_data['CD'] = data['CD']
req_data.head()

,d_votes,r_votes,CD
0,0.425584,0.508676,1
1,0.337908,0.594626,1
2,0.260996,0.682024,4
3,0.260911,0.684772,4
4,0.354584,0.582430,1


In [7]:
req_data['d_wasted'] = req_data['d_votes']
req_data['r_wasted'] = req_data['r_votes']
for index in range(len(req_data)):
    if req_data['d_votes'][index] > 0.5:
        req_data['d_wasted'][index] = req_data['d_votes'][index] - 0.5
        req_data['r_wasted'][index] = req_data['r_votes'][index]
        
    elif req_data['r_votes'][index] > 0.5:
        req_data['r_wasted'][index] = req_data['r_votes'][index] - 0.5
        req_data['d_wasted'][index] = req_data['d_votes'][index]
        

# req_data.loc[req_data['d_votes'] > 0.5]['d_wasted'] = req_data['d_votes'] - 0.5
# req_data.loc[req_data['r_votes'] > 0.5]['r_wasted'] = req_data['r_votes'] - 0.5
        
req_data.head(30)

<ipython-input-7-ec1cc5847aff>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['r_wasted'][index] = req_data['r_votes'][index] - 0.5
<ipython-input-7-ec1cc5847aff>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['d_wasted'][index] = req_data['d_votes'][index]
<ipython-input-7-ec1cc5847aff>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['d_wasted'][index] = req_data['d_votes'][index] - 0.5
<ipython-input-

,d_votes,r_votes,CD,d_wasted,r_wasted
0,0.425584,0.508676,1,0.425584,0.008676
1,0.337908,0.594626,1,0.337908,0.094626
2,0.260996,0.682024,4,0.260996,0.182024
3,0.260911,0.684772,4,0.260911,0.184772
4,0.354584,0.582430,1,0.354584,0.082430
5,0.372163,0.564888,1,0.372163,0.064888
6,0.354302,0.580062,4,0.354302,0.080062
7,0.250248,0.696930,3,0.250248,0.196930
8,0.430586,0.493156,2,0.430586,0.493156
9,0.314637,0.630601,4,0.314637,0.130601


In [8]:
fairness_metric(req_data)

0.3149161925437755

In [9]:
print(len(req_data[req_data['CD'] == 4]))
fairness_metric(req_data[req_data['CD'] == 4])

39


E:\IIT\3rd Sem\MATH 497\Algorithmic-Redistricting\Fairness_Metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['d_wasted'] = data['d_wasted'] - shift
E:\IIT\3rd Sem\MATH 497\Algorithmic-Redistricting\Fairness_Metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['r_wasted'] = data['r_wasted'] - shift


0.4182282611807085

In [10]:
print(len(req_data[req_data['CD'] == 3]))
fairness_metric(req_data[req_data['CD'] == 3])

16


E:\IIT\3rd Sem\MATH 497\Algorithmic-Redistricting\Fairness_Metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['d_wasted'] = data['d_wasted'] - shift
E:\IIT\3rd Sem\MATH 497\Algorithmic-Redistricting\Fairness_Metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['r_wasted'] = data['r_wasted'] - shift


0.3536340745845218

In [11]:
print(len(req_data[req_data['CD'] == 2]))
fairness_metric(req_data[req_data['CD'] == 2])

24


E:\IIT\3rd Sem\MATH 497\Algorithmic-Redistricting\Fairness_Metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['d_wasted'] = data['d_wasted'] - shift
E:\IIT\3rd Sem\MATH 497\Algorithmic-Redistricting\Fairness_Metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['r_wasted'] = data['r_wasted'] - shift


0.4115704972257607

In [12]:
print(len(req_data[req_data['CD'] == 1]))
fairness_metric(req_data[req_data['CD'] == 1])

20


E:\IIT\3rd Sem\MATH 497\Algorithmic-Redistricting\Fairness_Metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['d_wasted'] = data['d_wasted'] - shift
E:\IIT\3rd Sem\MATH 497\Algorithmic-Redistricting\Fairness_Metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['r_wasted'] = data['r_wasted'] - shift


0.08938723344244148

In [13]:
def myCustomLoss(my_outputs, my_labels):
    
    
    
    return fairness_metric(my_outputs)

In [14]:
class DQN(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(in_features = 4 , out_features = 24)   
        self.fc2 = nn.Linear(in_features = 24, out_features = 32)
        self.out = nn.Linear(in_features = 32, out_features = 4)

#         self.optimizer = optim.Adam(self.parameters(), lr=lr)
#         self.loss = nn.MSELoss()
#         self.device = T.device('cuda' if T.cuda.is_available() else 'cpu')
#         self.to(self.device)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.out(x)

        return actions

In [15]:
Experience = namedtuple(
    'Experience',
    ('state', 'action', 'next_state', 'reward')
)

In [16]:
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory[self.push_count % self.capacity] = experience
        self.push_count += 1
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def can_provide_sample(self, batch_size):
        return len(self.memory) >= batch_size

In [17]:
class EpsilonGreedyStrategy():
    def __init__(self, start, end, decay):
        self.start = start
        self.end = end
        self.decay = decay
        
    def get_exploration_rate(self, current_step):
        return self.end + (self.start - self.end) * \
            math.exp(-1. * current_step * self.decay)

In [18]:
class Agent():
    def __init__(self, strategy, num_actions, device):
        self.current_step = 0
        self.strategy = strategy
        self.num_actions = num_actions
        self.device = device
        
    def select_action(self, state, policy_net):
        rate = self.strategy.get_exploration_rate(self.current_step)
        self.current_step += 1

        if rate > random.random():
            action = random.randrange(self.num_actions)
            return torch.tensor([action]).to(self.device) # explore      
        else:
            with torch.no_grad():
                return policy_net(state).argmax(dim=1).to(self.device) # exploit

In [19]:
batch_size = 256
gamma = 0.999
eps_start = 1
eps_end = 0.01
eps_decay = 0.001
target_update = 10
memory_size = 100000
lr = 0.001
num_episodes = 500

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:

strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay)

In [28]:
agent = Agent(strategy, 4, device)
memory = ReplayMemory(memory_size)

In [29]:
policy_net = DQN().to(device)
target_net = DQN().to(device)

In [30]:
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

DQN(
  (fc1): Linear(in_features=4, out_features=24, bias=True)
  (fc2): Linear(in_features=24, out_features=32, bias=True)
  (out): Linear(in_features=32, out_features=4, bias=True)
)

In [31]:
optimizer = optim.Adam(params=policy_net.parameters(), lr=lr)

In [32]:
def extract_tensors(experiences):
    # Convert batch of Experiences to Experience of batches
    batch = Experience(*zip(*experiences))

    t1 = torch.cat(batch.state)
    t2 = torch.cat(batch.action)
    t3 = torch.cat(batch.reward)
    t4 = torch.cat(batch.next_state)

    return (t1,t2,t3,t4)